<a href="https://colab.research.google.com/github/DayenaJeong/FS_neuron/blob/main/MNIST_CIFAR10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

MNIST 다운로드

In [2]:
# MNIST 데이터셋의 변환 정의
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # MNIST는 흑백 이미지이므로 채널이 하나
])

# 훈련 및 테스트 데이터셋 로딩
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 데이터 로더 설정
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 9912422/9912422 [00:00<00:00, 16374211.48it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 28881/28881 [00:00<00:00, 540434.51it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:00<00:00, 4520355.91it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 4542/4542 [00:00<00:00, 4751940.33it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



CIFAR10 다운로드

In [19]:
# CIFAR-10 데이터셋의 변환 정의
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 훈련 및 테스트 데이터셋 로딩
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 클래스 필터링: 'airplane', 'automobile', 'bird'는 각각 0, 1, 2 인덱스
classes = [0, 1, 2]
train_indices = [i for i, (img, label) in enumerate(train_data) if label in classes]
test_indices = [i for i, (img, label) in enumerate(test_data) if label in classes]

# 데이터셋 필터링
filtered_train_data = Subset(train_data, train_indices)
filtered_test_data = Subset(test_data, test_indices)

# 데이터 로더 설정
train_loader = DataLoader(filtered_train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(filtered_test_data, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:03<00:00, 43009457.31it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


MNIST 분류

간단한 MLP

In [16]:
class Swish(nn.Module):
    def __init__(self, beta=1.0):
        super(Swish, self).__init__()
        self.beta = beta  # β is a learnable parameter

    def forward(self, x):
        return x * torch.sigmoid(self.beta * x)

In [18]:
# 간단한 신경망 정의
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)  # MNIST 이미지는 28x28 크기
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)  # MNIST에는 10개의 클래스
        self.swish = Swish()  # Swish 활성화 함수 인스턴스화

    def forward(self, x):
        x = x.view(-1, 28*28)  # 평탄화
        x = self.swish(self.fc1(x))
        x = self.swish(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델, 손실 함수, 옵티마이저 초기화
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 정확도 계산 함수
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

# 모델 훈련 및 평가 함수
def train_and_evaluate(model, criterion, optimizer, train_loader, test_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        test_loss = 0.0
        test_acc = 0.0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                test_acc += accuracy(outputs, labels)

        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Test Loss: {test_loss}, Test Accuracy: {test_acc}%')

# 모델 훈련 및 평가
train_and_evaluate(model, criterion, optimizer, train_loader, test_loader)

Epoch 1, Loss: 0.2727195942488465, Test Loss: 0.13633221003840898, Test Accuracy: 95.53144904458598%
Epoch 2, Loss: 0.11955884799484347, Test Loss: 0.10830800770930234, Test Accuracy: 96.6062898089172%
Epoch 3, Loss: 0.09106782336792807, Test Loss: 0.0895881741050005, Test Accuracy: 97.29299363057325%
Epoch 4, Loss: 0.07376280576358044, Test Loss: 0.09580431943355205, Test Accuracy: 97.07404458598727%
Epoch 5, Loss: 0.062447778848823965, Test Loss: 0.09465180829524769, Test Accuracy: 97.15366242038216%
Epoch 6, Loss: 0.054923507267609736, Test Loss: 0.07566458663653292, Test Accuracy: 97.9000796178344%
Epoch 7, Loss: 0.047483977689623716, Test Loss: 0.11004975775296053, Test Accuracy: 97.04418789808918%
Epoch 8, Loss: 0.042082396991604785, Test Loss: 0.0707861085081572, Test Accuracy: 98.06926751592357%
Epoch 9, Loss: 0.037818536453119944, Test Loss: 0.0863741654705736, Test Accuracy: 97.61146496815287%
Epoch 10, Loss: 0.03409840310567559, Test Loss: 0.08198519209718663, Test Accuracy:

FS Swish

In [8]:
# Implementation of spike function for PyTorch custom gradient
class SpikeFunction(autograd.Function):
    @staticmethod
    def forward(ctx, v_scaled):
        z_ = torch.where(v_scaled > 0, torch.ones_like(v_scaled), torch.zeros_like(v_scaled))
        ctx.save_for_backward(v_scaled)
        return z_

    @staticmethod
    def backward(ctx, grad_output):
        v_scaled, = ctx.saved_tensors
        dz_dv_scaled = torch.maximum(1 - torch.abs(v_scaled), torch.tensor(0.0, device=v_scaled.device))
        dE_dv_scaled = grad_output * dz_dv_scaled
        return dE_dv_scaled

# Call spike function for PyTorch
def spike_function(v_scaled):
    return SpikeFunction.apply(v_scaled)

k=4로 했을 때

In [9]:
class FSSwish(nn.Module):
    def __init__(self):
        super(FSSwish, self).__init__()
        self.h = torch.tensor([5.9128, 2.9864, 1.4755, 0.4429], dtype=torch.float32)
        self.d = torch.tensor([6.0788, 3.0843, 1.5167, 0.7723], dtype=torch.float32)
        self.T = torch.tensor([5.7271, 2.8642, 1.3162, 0.6181], dtype=torch.float32)
        self.K = len(self.h)

    def forward(self, x):
        v = x

        # Initialize temporary output for FS spike neural network
        temp_out = torch.zeros_like(v)

        # Implement FS spike neural network
        for t in range(len(self.T)):
            v_scaled = (v - self.T[t]) / (torch.abs(v) + 1)
            z = spike_function(v_scaled)
            temp_out += z * self.d[t]
            v = v - z * self.h[t]

        return temp_out

In [11]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        self.fs_swish1 = FSSwish()
        self.fs_swish2 = FSSwish()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fs_swish1(self.fc1(x))
        x = self.fs_swish2(self.fc2(x))
        x = self.fc3(x)
        return x

# 데이터 로딩 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 모델, 손실 함수, 옵티마이저 초기화
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 정확도 계산 함수
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

# 모델 훈련 및 평가 함수
def train_and_evaluate(model, criterion, optimizer, train_loader, test_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        test_loss = 0.0
        test_acc = 0.0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                test_acc += accuracy(outputs, labels)

        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Test Loss: {test_loss}, Test Accuracy: {test_acc}%')

# 모델 훈련 및 평가
train_and_evaluate(model, criterion, optimizer, train_loader, test_loader)

Epoch 1, Loss: 0.29050124780153797, Test Loss: 0.1319344721713762, Test Accuracy: 95.72054140127389%
Epoch 2, Loss: 0.13319084876334109, Test Loss: 0.09869940609783884, Test Accuracy: 96.77547770700637%
Epoch 3, Loss: 0.09770265747177433, Test Loss: 0.09757748859208581, Test Accuracy: 97.15366242038216%
Epoch 4, Loss: 0.08069958772807162, Test Loss: 0.11313831546741678, Test Accuracy: 96.55652866242038%
Epoch 5, Loss: 0.06902846806312302, Test Loss: 0.07179615582206181, Test Accuracy: 97.70103503184713%
Epoch 6, Loss: 0.058739957208660785, Test Loss: 0.08771366003396917, Test Accuracy: 97.53184713375796%
Epoch 7, Loss: 0.051852413685340784, Test Loss: 0.10501125653096027, Test Accuracy: 96.94466560509554%
Epoch 8, Loss: 0.04646297352019149, Test Loss: 0.0982094240055277, Test Accuracy: 97.20342356687898%
Epoch 9, Loss: 0.04094309258999041, Test Loss: 0.08422336409036753, Test Accuracy: 97.57165605095541%
Epoch 10, Loss: 0.035407797746131446, Test Loss: 0.08500293303375796, Test Accurac

k=16로 했을 때

In [12]:
class FSSwish(nn.Module):
    def __init__(self):
        super(FSSwish, self).__init__()
        self.h = torch.tensor([-0.3462,  1.7540,  0.7608,  2.2101,  2.2010,  2.0671,  1.5055,  0.9358,
0.7226,  0.5611,  0.4563,  0.3873,  0.3402,  0.3063,  0.2518,  0.2383], dtype=torch.float32)
        self.d = torch.tensor([-0.1988,  1.8982,  0.5589,  2.2857,  2.2202,  2.0731,  1.5142,  0.9145,
0.6390,  0.4360,  0.3265,  0.2403,  0.1830,  0.1436,  0.1209,  0.1377], dtype=torch.float32)
        self.T = torch.tensor([-3.7520,  3.2999,  0.7893,  2.3260,  2.3061,  2.1326,  1.5841,  0.9676,
0.7256,  0.5612,  0.4561,  0.3868,  0.3397,  0.3061,  0.2929,  0.2950], dtype=torch.float32)
        self.K = len(self.h)

    def forward(self, x):
        v = x

        # Initialize temporary output for FS spike neural network
        temp_out = torch.zeros_like(v)

        # Implement FS spike neural network
        for t in range(len(self.T)):
            v_scaled = (v - self.T[t]) / (torch.abs(v) + 1)
            z = spike_function(v_scaled)
            temp_out += z * self.d[t]
            v = v - z * self.h[t]

        return temp_out

In [13]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        self.fs_swish1 = FSSwish()
        self.fs_swish2 = FSSwish()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fs_swish1(self.fc1(x))
        x = self.fs_swish2(self.fc2(x))
        x = self.fc3(x)
        return x

# 데이터 로딩 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 모델, 손실 함수, 옵티마이저 초기화
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 정확도 계산 함수
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

# 모델 훈련 및 평가 함수
def train_and_evaluate(model, criterion, optimizer, train_loader, test_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        test_loss = 0.0
        test_acc = 0.0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                test_acc += accuracy(outputs, labels)

        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Test Loss: {test_loss}, Test Accuracy: {test_acc}%')

# 모델 훈련 및 평가
train_and_evaluate(model, criterion, optimizer, train_loader, test_loader)

Epoch 1, Loss: 0.2753205719564769, Test Loss: 0.14593418538985645, Test Accuracy: 95.49164012738854%
Epoch 2, Loss: 0.12237779358560756, Test Loss: 0.13645759765740934, Test Accuracy: 95.80015923566879%
Epoch 3, Loss: 0.09215622913610659, Test Loss: 0.09098237098133856, Test Accuracy: 97.19347133757962%
Epoch 4, Loss: 0.07403043022866784, Test Loss: 0.09138082898133672, Test Accuracy: 97.02428343949045%
Epoch 5, Loss: 0.06176981396962211, Test Loss: 0.08959078332438923, Test Accuracy: 97.20342356687898%
Epoch 6, Loss: 0.0565467014352653, Test Loss: 0.10024597880409758, Test Accuracy: 96.96457006369427%
Epoch 7, Loss: 0.04865642305435504, Test Loss: 0.08366975214559982, Test Accuracy: 97.46218152866243%
Epoch 8, Loss: 0.04585914536558493, Test Loss: 0.1111543726326038, Test Accuracy: 96.89490445859873%
Epoch 9, Loss: 0.04037120858875244, Test Loss: 0.08179352888158985, Test Accuracy: 97.75079617834395%
Epoch 10, Loss: 0.03676919062653043, Test Loss: 0.08529485990911086, Test Accuracy: 9

CIFAR10 분류

기존 swish

In [20]:
# 신경망 정의
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 3)  # 클래스 수 3
        self.swish = Swish()  # Swish 활성화 함수 인스턴스화

    def forward(self, x):
        x = x.view(-1, 3*32*32)  # 평탄화
        x = self.swish(self.fc1(x))
        x = self.swish(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델, 손실 함수, 옵티마이저 초기화
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 정확도 계산 함수
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

# 모델 훈련 및 평가 함수
def train_and_evaluate(model, criterion, optimizer, train_loader, test_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        test_loss = 0.0
        test_acc = 0.0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                test_acc += accuracy(outputs, labels)

        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Test Loss: {test_loss}, Test Accuracy: {test_acc}%')

# 모델 훈련 및 평가
train_and_evaluate(model, criterion, optimizer, train_loader, test_loader)

Epoch 1, Loss: 0.6542027058753561, Test Loss: 0.5580013565560604, Test Accuracy: 78.74715045592706%
Epoch 2, Loss: 0.5096942858493074, Test Loss: 0.48384127464700255, Test Accuracy: 80.80357142857143%
Epoch 3, Loss: 0.43036826031005126, Test Loss: 0.4990546056564818, Test Accuracy: 79.8109802431611%
Epoch 4, Loss: 0.37340445917971593, Test Loss: 0.5056905971562609, Test Accuracy: 81.24525075987842%
Epoch 5, Loss: 0.3200741209882371, Test Loss: 0.4962940942099754, Test Accuracy: 82.09536474164133%
Epoch 6, Loss: 0.2827190731117066, Test Loss: 0.5119229301493219, Test Accuracy: 81.38772796352583%
Epoch 7, Loss: 0.23490818456766452, Test Loss: 0.5415984772621317, Test Accuracy: 82.04312310030396%
Epoch 8, Loss: 0.20509039376010285, Test Loss: 0.6256535573208586, Test Accuracy: 81.25%
Epoch 9, Loss: 0.18551685756191294, Test Loss: 0.6219574333505428, Test Accuracy: 81.94338905775076%
Epoch 10, Loss: 0.1639283260607973, Test Loss: 0.684867446726941, Test Accuracy: 81.13126899696049%


k=4일때

In [22]:
class FSSwish(nn.Module):
    def __init__(self):
        super(FSSwish, self).__init__()
        self.h = torch.tensor([5.9128, 2.9864, 1.4755, 0.4429], dtype=torch.float32)
        self.d = torch.tensor([6.0788, 3.0843, 1.5167, 0.7723], dtype=torch.float32)
        self.T = torch.tensor([5.7271, 2.8642, 1.3162, 0.6181], dtype=torch.float32)
        self.K = len(self.h)

    def forward(self, x):
        v = x

        # Initialize temporary output for FS spike neural network
        temp_out = torch.zeros_like(v)

        # Implement FS spike neural network
        for t in range(len(self.T)):
            v_scaled = (v - self.T[t]) / (torch.abs(v) + 1)
            z = spike_function(v_scaled)
            temp_out += z * self.d[t]
            v = v - z * self.h[t]

        return temp_out

In [ ]:
class FSSwish(nn.Module):
    def __init__(self):
        super(FSSwish, self).__init__()
        self.h = torch.tensor([5.9128, 2.9864, 1.4755, 0.4429], dtype=torch.float32)
        self.d = torch.tensor([6.0788, 3.0843, 1.5167, 0.7723], dtype=torch.float32)
        self.T = torch.tensor([5.7271, 2.8642, 1.3162, 0.6181], dtype=torch.float32)
        self.K = len(self.h)

    def forward(self, x):
        v = x

        # Initialize temporary output for FS spike neural network
        temp_out = torch.zeros_like(v)

        # Implement FS spike neural network
        for t in range(len(self.T)):
            v_scaled = (v - self.T[t]) / (torch.abs(v) + 1)
            z = spike_function(v_scaled)
            temp_out += z * self.d[t]
            v = v - z * self.h[t]

        return temp_out

In [23]:
# 신경망 정의
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 3)  # 클래스 수 3
        self.fs_swish1 = FSSwish()
        self.fs_swish2 = FSSwish()

    def forward(self, x):
        x = x.view(-1, 3*32*32)  # 평탄화
        x = self.fs_swish1(self.fc1(x))
        x = self.fs_swish2(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델, 손실 함수, 옵티마이저 초기화
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 정확도 계산 함수
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

# 모델 훈련 및 평가 함수
def train_and_evaluate(model, criterion, optimizer, train_loader, test_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        test_loss = 0.0
        test_acc = 0.0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                test_acc += accuracy(outputs, labels)

        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Test Loss: {test_loss}, Test Accuracy: {test_acc}%')

# 모델 훈련 및 평가
train_and_evaluate(model, criterion, optimizer, train_loader, test_loader)

Epoch 1, Loss: 0.6810605652788853, Test Loss: 0.5636912000940201, Test Accuracy: 77.65007598784194%
Epoch 2, Loss: 0.5287138727117092, Test Loss: 0.5294463869104994, Test Accuracy: 78.14399696048632%
Epoch 3, Loss: 0.4478860357974438, Test Loss: 0.48209420607445086, Test Accuracy: 80.90805471124621%
Epoch 4, Loss: 0.38907484494625255, Test Loss: 0.4741644060358088, Test Accuracy: 82.1048632218845%
Epoch 5, Loss: 0.3445899662185223, Test Loss: 0.48440222790900694, Test Accuracy: 81.33073708206686%
Epoch 6, Loss: 0.28760026236797903, Test Loss: 0.520010549337306, Test Accuracy: 80.28115501519757%
Epoch 7, Loss: 0.24836368678098028, Test Loss: 0.5151726020143387, Test Accuracy: 80.12917933130699%
Epoch 8, Loss: 0.21059976343778855, Test Loss: 0.5500455796718597, Test Accuracy: 81.08377659574468%
Epoch 9, Loss: 0.18493643533042137, Test Loss: 0.5761560956214337, Test Accuracy: 82.1523556231003%
Epoch 10, Loss: 0.1686219625809091, Test Loss: 0.6144288137872168, Test Accuracy: 81.15026595744

k=16일때

In [24]:
class FSSwish(nn.Module):
    def __init__(self):
        super(FSSwish, self).__init__()
        self.h = torch.tensor([-0.3462,  1.7540,  0.7608,  2.2101,  2.2010,  2.0671,  1.5055,  0.9358,
0.7226,  0.5611,  0.4563,  0.3873,  0.3402,  0.3063,  0.2518,  0.2383], dtype=torch.float32)
        self.d = torch.tensor([-0.1988,  1.8982,  0.5589,  2.2857,  2.2202,  2.0731,  1.5142,  0.9145,
0.6390,  0.4360,  0.3265,  0.2403,  0.1830,  0.1436,  0.1209,  0.1377], dtype=torch.float32)
        self.T = torch.tensor([-3.7520,  3.2999,  0.7893,  2.3260,  2.3061,  2.1326,  1.5841,  0.9676,
0.7256,  0.5612,  0.4561,  0.3868,  0.3397,  0.3061,  0.2929,  0.2950], dtype=torch.float32)
        self.K = len(self.h)

    def forward(self, x):
        v = x

        # Initialize temporary output for FS spike neural network
        temp_out = torch.zeros_like(v)

        # Implement FS spike neural network
        for t in range(len(self.T)):
            v_scaled = (v - self.T[t]) / (torch.abs(v) + 1)
            z = spike_function(v_scaled)
            temp_out += z * self.d[t]
            v = v - z * self.h[t]

        return temp_out

In [25]:
# 신경망 정의
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(3*32*32, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 3)  # 클래스 수 3
        self.fs_swish1 = FSSwish()
        self.fs_swish2 = FSSwish()

    def forward(self, x):
        x = x.view(-1, 3*32*32)  # 평탄화
        x = self.fs_swish1(self.fc1(x))
        x = self.fs_swish2(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델, 손실 함수, 옵티마이저 초기화
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 정확도 계산 함수
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

# 모델 훈련 및 평가 함수
def train_and_evaluate(model, criterion, optimizer, train_loader, test_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        test_loss = 0.0
        test_acc = 0.0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                test_acc += accuracy(outputs, labels)

        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Test Loss: {test_loss}, Test Accuracy: {test_acc}%')

# 모델 훈련 및 평가
train_and_evaluate(model, criterion, optimizer, train_loader, test_loader)

Epoch 1, Loss: 0.6573027565124187, Test Loss: 0.556056982025187, Test Accuracy: 76.92819148936171%
Epoch 2, Loss: 0.5101463003361478, Test Loss: 0.5050672641459931, Test Accuracy: 79.41679331306992%
Epoch 3, Loss: 0.43620246192242235, Test Loss: 0.533622716969632, Test Accuracy: 79.90121580547113%
Epoch 4, Loss: 0.38436375932490574, Test Loss: 0.5088746687199207, Test Accuracy: 80.68484042553192%
Epoch 5, Loss: 0.3197494422818752, Test Loss: 0.48962205775240636, Test Accuracy: 81.78191489361703%
Epoch 6, Loss: 0.2754131350111454, Test Loss: 0.526522511497457, Test Accuracy: 80.54711246200608%
Epoch 7, Loss: 0.2432443140986118, Test Loss: 0.5358133639426942, Test Accuracy: 80.1006838905775%
Epoch 8, Loss: 0.20029366594996859, Test Loss: 0.580719438005001, Test Accuracy: 82.3233282674772%
Epoch 9, Loss: 0.17168254760351587, Test Loss: 0.634213370845673, Test Accuracy: 81.35448328267476%
Epoch 10, Loss: 0.15401994945838096, Test Loss: 0.6723577988908646, Test Accuracy: 81.39247720364742%
